# BE - Machine Learning

### Données

Le dataset est un dataset UCML, on pourrait utiliser le module *ucmlrepo* et la methode *fetch_ucirepo* associée, mais le chargement des données est long (~2 minutes). On privilégiera donc un téléchargement et des méthodes de lecture manuelles, les données ne faisant que 600Ko

Le dataset a été construit pour prédire la target *income* qui comporte deux valeurs (>50K, <=50K) à partir de 14 features. Cette construction en "classes" nous impose de travailler avec des classifiers plutôt qu'avec des regressions.

In [2]:
import pandas as pd

names = pd.read_table("data/adult.names", sep=":", skiprows=96, header=None)[0].tolist()
names.append("income") # La target n'est pas dans le fichier des noms
adult = pd.read_csv("data/adult.data", sep=", ", names=names, engine='python', index_col=False) # Le engine permet juste d'éviter un warning

### Nettoyage et Pre processing

Avant de mettre en place des modèles il nous fait nettoyer un peu les données pour pouvoir les exploiter. En particulier, nous aimerions supprimer les *valuers manquantes* ainsi que les *outliers*. En mettant en place les algo classiques, nous nous sommes vite rendu compte d'un problème : la plupart des catégories sont encodées sous forme de texte. Ainsi les algo ne peuvent fonctionner. C'est la raison pour laquelle nous allons mettre en place cette pipeline de nettoyage :
- **Suppression des #NA :** Les NA étant encodés par des "?" (il n'y a pas de NA sinon) nous allons supprimer chaque lignes qui en comportent
- **Encodage des colonnes :** Nous allons encoder chaque colonnes de la façon suivante : chaque valeur unique se voit attribuer un entier. Nous n'avons que peu de recul pour l'instant et ne savons pas si c'est pertinent
- **Normalisation :** POur contrer l'effet du nombre e colonnes, nous allons normaliser celles ci


In [3]:
#### Suppression des NA ####
adult.replace("?", pd.NA, inplace=True)  # Les NA sont codés par des "?"

for key in adult.keys():
    n_na = adult[key].isna().sum()
    if n_na > 0:
        print(f"{key} : {n_na} NA")

adult.dropna(inplace=True)  # On supprime les lignes avec NA

workclass : 1836 NA
occupation : 1843 NA
native-country : 583 NA


In [4]:
#### Encodage des colonnes ####
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column in adult.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    adult[column] = le.fit_transform(adult[column])
    label_encoders[column] = le

In [5]:
#### Normalisation ####
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
for column in adult.columns:
    if column != "income":  # On ne normalise pas la target
        adult[[column]] = scaler.fit_transform(adult[[column]])

### Data Vis

Projet : faire une acp ? regarder un peu comment on se sort des 15 variables degeus

### Tests sur les outliers ?

In [6]:
from sklearn.ensemble import IsolationForest

print(f"Nombre de lignes après suppression des NA : {adult.shape[0]}")
iso_forest = IsolationForest(contamination=0.1, random_state=42)
outliers = iso_forest.fit_predict(adult.drop(columns=["income"]))
mask = outliers != -1
adult = adult[mask]

print(f"Nombre de lignes après suppression des outliers : {adult.shape[0]}")

Nombre de lignes après suppression des NA : 30162
Nombre de lignes après suppression des outliers : 27145


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X = adult.drop(columns=["income"])
y = adult["income"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90      4141
           1       0.72      0.60      0.66      1288

    accuracy                           0.85      5429
   macro avg       0.80      0.76      0.78      5429
weighted avg       0.84      0.85      0.85      5429



In [9]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'seed': 42
}

bst = xgb.train(params, dtrain, num_boost_round=100)
y_pred = bst.predict(dtest)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred] 

print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91      4141
           1       0.75      0.64      0.69      1288

    accuracy                           0.87      5429
   macro avg       0.82      0.79      0.80      5429
weighted avg       0.86      0.87      0.86      5429

